In [2]:
# Notebook finds distribution of assays after resampling before training/test split

In [3]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns

In [4]:
from sklearn.linear_model import LogisticRegression
from io import StringIO
import sys
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold

In [5]:
all_assay_all_compounds =pd.read_csv("Predictions_train_heldout_scaled_prob_all_assays.csv")
all_assay_all_compounds

,assay,StdInChI,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,ts,pc,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled
0,588458,"InChI=1S/C19H19N3O5S3/c23-16-6-5-13(30(25,26)2...",0.215985,0,0.278423,0.291141,0,0.306456,0.0,0.418605,0.430046,Training,True,True,0.387873,0.475014
1,588458,InChI=1S/C9H8ClN3S/c1-6-11-12-9(14)13(6)8-4-2-...,0.240910,0,0.278423,0.253367,0,0.306456,0.0,0.409091,0.680534,Training,True,True,0.432633,0.413382
2,588458,InChI=1S/C20H20N2O4/c1-2-25-20(24)17-12-18-16(...,0.232871,0,0.278423,0.154150,0,0.306456,0.0,0.447368,0.701574,Training,True,True,0.418196,0.251504
3,588458,InChI=1S/C26H24N2O5S/c1-33-19-11-9-18(10-12-19...,0.142133,0,0.278423,0.359792,1,0.306456,1.0,0.341463,0.453313,Training,False,True,0.255247,0.538452
4,588458,InChI=1S/C19H21N3O3S/c1-13-10-16-11-15(4-9-19(...,0.253998,0,0.278423,0.202499,0,0.306456,0.0,0.388889,0.646760,Training,True,True,0.456137,0.330388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50467,1117305,InChI=1S/C12H9F3N2O2/c1-7-10(6-16-19-7)11(18)1...,0.340631,0,0.357222,0.357490,1,0.330242,0.0,0.296296,0.479437,HeldOut,True,False,0.476778,0.520342
50468,1117305,InChI=1S/C12H10N2/c1-8-12-10(6-7-13-8)9-4-2-3-...,0.242578,0,0.357222,0.244216,0,0.330242,0.0,0.363636,0.619949,HeldOut,True,True,0.339534,0.369752
50469,1117305,InChI=1S/C8H4Cl2N2O2/c9-3-1-5-6(2-4(3)10)12-8(...,0.403774,1,0.357222,0.352388,1,0.330242,1.0,0.333333,0.541732,HeldOut,True,True,0.536212,0.516533
50470,1117305,InChI=1S/C15H12O5/c16-9-3-1-8(2-4-9)13-7-12(19...,0.478660,1,0.357222,0.243638,0,0.330242,0.0,0.444444,0.506977,HeldOut,False,True,0.594464,0.368877


In [6]:
len(all_assay_all_compounds.assay.unique())

92

In [7]:
stats_list=[]
for assay in all_assay_all_compounds.assay.unique():   
    per_assay_compounds=all_assay_all_compounds[all_assay_all_compounds["assay"]==assay]
    inactives= per_assay_compounds["true"].value_counts()[0]
    actives= per_assay_compounds["true"].value_counts()[1]
    total=inactives+actives
    
    row=[assay, total, actives, inactives]
    stats_list.append(row)

In [11]:
df = pd.DataFrame(stats_list, columns=["assay", "total", "actives", "inactives"])
df["ratio"] = df["actives"]/df["total"]
df

,assay,total,actives,inactives,ratio
0,588458,448,112,336,0.250000
1,588334,528,132,396,0.250000
2,2642,764,191,573,0.250000
3,2156,568,142,426,0.250000
4,2330,328,82,246,0.250000
...,...,...,...,...,...
87,720504,128,96,32,0.750000
88,720532,301,210,91,0.697674
89,1159524,880,220,660,0.250000
90,1117304,386,126,260,0.326425


In [12]:
df.describe()

,total,actives,inactives,ratio
count,92.000000,92.000000,92.000000,92.000000
mean,548.608696,190.869565,357.739130,0.386139
std,557.909047,201.072029,404.245053,0.187110
min,44.000000,25.000000,11.000000,0.250000
25%,157.250000,75.000000,93.250000,0.250000
50%,416.000000,128.500000,253.500000,0.265181
75%,707.000000,226.250000,444.750000,0.492805
max,3416.000000,1224.000000,2562.000000,0.750000


In [9]:
df.to_csv("assay_compounds_stats_list_bothsideundersampling.csv", index=False)